In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import seaborn as sns
from matplotlib import rcParams

from sklearn.ensemble import RandomForestClassifier

import datetime

pd.set_option('display.max_columns',50)
pd.set_option('display.max_rows',50)

%matplotlib inline

## Sites to look at

https://www.dataquest.io/blog/kaggle-tutorial/

## Exploration

In [ ]:
#Load dataset
df = pd.read_csv('../../training_set_VU_DM_2014.csv')

In [ ]:
#convert date_time to datetime
df["date_time"] = pd.to_datetime(df["date_time"])

#add column for the starting date of the booking
df["book_start"] = df["date_time"] + pd.to_timedelta(df['srch_booking_window'], unit='D')

#extract month
df["book_start_month"] = df["book_start"].dt.month

In [ ]:
df.dtypes

In [ ]:
#should not use columns only available in training set
df.drop(["position","click_bool","gross_bookings_usd"],axis=1,inplace=True)
#keep booking_bool, as this is our target!
#can also drop random_bool, as this influences the sorting but we don't have position!
df.drop(["random_bool"],axis=1,inplace=True)

In [ ]:
#Take minimum of comp#_rate columns. If Expedia is cheaper than all competitors (all are 1), this will be 1.
#If one competitor is cheaper (-1), this column will equal -1 (and have less chance of being booked at Expedia!)
df["comp"] = df[["comp%d_rate"%i for i in range(1,9)]].min(axis=1)

In [ ]:
#multiply comp_rate and comp_rate_percent_diff and drop the old columns
for i in range(1,9):
    df["comp%d"%i] = df["comp%d_rate"%i] * df["comp%d_rate_percent_diff"%i]
    df.drop(["comp%d_rate"%i, "comp%d_rate_percent_diff"%i],axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
#show correlations with the target booking_bool
df.corr()["booking_bool"]

In [ ]:
# #what are -1 values? Expedia has no availability, but competitor does? Why would Expedia then show the hotel?
# #(Create new feature based on availablility?)
# for i in range(1,9):
#     print(df["comp%d_inv"%i].value_counts())
#     print("Number of NaNs: %d"%df["comp%d_inv"%i].isnull().sum())

# Exploration

In [ ]:
df.head(50)

In [ ]:
print(df.columns.values)

In [ ]:
df.dtypes

In [ ]:
#show hist of all columns
df.hist(df.columns.values,figsize=(8,10*len(df.columns)),layout=(len(df.columns),1));